In [1]:
from keras.layers import Input, Dense, Conv1D, Flatten, MaxPooling1D
from keras.models import Model, Sequential

def network(input_window_size=60, filter_number=32, conv_window=(3,), pooling_window=(2,), activation='relu', 
            dense_activation='softmax', optimizer='adam', loss='categorical_crossentropy', layers=1):
    model = Sequential()
    
    # Input Layer
    model.add(Conv1D(filter_number, conv_window, activation=activation, padding='valid', input_shape=(input_window_size, 1)))
    model.add(MaxPooling1D(pooling_window, padding='same'))
    
    # Hidden Layers
    filter_number_temp = filter_number
    for i in range(layers):
        filter_number_temp = filter_number_temp *  2
        model.add(Conv1D(filter_number_temp, conv_window, activation=activation, padding='valid'))
        model.add(MaxPooling1D(pooling_window, padding='same'))
    
    # Output Layer
    model.add(Flatten())
    model.add(Dense(2, activation=dense_activation))

    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy', 'categorical_accuracy'])

    return model

Using TensorFlow backend.


In [2]:
import os
import h5py
import numpy as np
from keras.utils import to_categorical

size = 60

base_path = '/home/irene/Documents/Datos-Tesis/Training_window_station_name'

total_files = len(os.listdir(base_path))
x_dataset = np.zeros(shape=(0, size))
y_dataset = np.zeros(shape=(0, 1))
count = 1
skipped = []
for file_path in os.listdir(base_path):
    current_index = 0
    
    h5f = h5py.File(os.path.join(base_path, file_path), 'r')
    
    while current_index < len(h5f['normal'][:]):
        if current_index + size > len(h5f['normal'][:]):
            difference = len(h5f['normal'][:]) - current_index
            x = h5f['normal'][current_index:current_index+difference]
            y = h5f['transformed'][current_index:current_index+difference]
            difference2 = size - difference
            x = np.append(x, np.zeros(difference2))
            y = [np.amax(y)]
        else:
            x = h5f['normal'][current_index:current_index+size]
            y = h5f['transformed'][current_index:current_index+size]
            y = [np.amax(y)]

        x_dataset = np.vstack((x_dataset, x))
        y_dataset = np.vstack((y_dataset, y))
        
        current_index = current_index + 1
    
    print(count)
    count = count + 1

x_dataset = np.reshape(x_dataset, (len(x_dataset), size, 1))
y_dataset = np.reshape(y_dataset, (len(y_dataset), 1))
y_dataset = to_categorical(y_dataset)

# indices = np.arange(x_dataset.shape[0])
# np.random.shuffle(indices)
# x_dataset= x_dataset[indices] 
# y_dataset= y_dataset[indices]

num_to_load = len(x_dataset)
split = int(num_to_load * 0.85)

x_train = x_dataset[0:split]
y_train = y_dataset[0:split]
x_test = x_dataset[split:]
y_test = y_dataset[split:]
    
# classifier.save_weights('/home/irene/Documents/Datos-Tesis/weights.h5')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152


In [3]:
cnn = network(layers=2)
cnn.fit(x_train, y_train, epochs=60, batch_size=32)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/60
129329/129329 [==============================] - 38s 297us/step - loss: 0.1912 - acc: 0.9464 - categorical_accuracy: 0.9464
Epoch 2/60
129329/129329 [==============================] - 35s 270us/step - loss: 0.1659 - acc: 0.9526 - categorical_accuracy: 0.9526
Epoch 3/60
129329/129329 [==============================] - 33s 252us/step - loss: 0.1543 - acc: 0.9556 - categorical_accuracy: 0.9556
Epoch 4/60
129329/129329 [==============================] - 40s 311us/step - loss: 0.1463 - acc: 0.9573 - categorical_accuracy: 0.9573
Epoch 5/60
129329/129329 [==============================] - 34s 265us/step - loss: 0.1385 - acc: 0.9587 - categorical_accuracy: 0.9587
Epoch 6/60
129329/129329 [==============================] - 34s 259us/step - loss: 0.1321 - acc: 0.9600 - categorical_accuracy: 0.9600
Epoch 7/60
129329/129329 [==============================] - 38s 293us/

129329/129329 [==============================] - 32s 244us/step - loss: 0.0237 - acc: 0.9922 - categorical_accuracy: 0.9922
Epoch 58/60
129329/129329 [==============================] - 32s 245us/step - loss: 0.0228 - acc: 0.9926 - categorical_accuracy: 0.9926
Epoch 59/60
129329/129329 [==============================] - 32s 247us/step - loss: 0.0251 - acc: 0.9919 - categorical_accuracy: 0.9919
Epoch 60/60
129329/129329 [==============================] - 32s 246us/step - loss: 0.0235 - acc: 0.9924 - categorical_accuracy: 0.9924


In [4]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

In [35]:
import numpy as np
import matplotlib.pyplot as plt
from obspy.core import read, UTCDateTime
from obspy.io.sac.util import get_sac_reftime
from obspy.signal.filter import lowpass, bandpass, highpass


def picker_p_predict(file):
    p_is_defined = False
    tr = read(file)[0]

    tr.filter('bandpass', freqmin=1.0, freqmax=10.0, corners=4, zerophase=True)
    tr.normalize()
    if 'a' in tr.stats.sac and tr.stats.sac.a is not None: 
        picker_p = int(tr.stats.sac.a * tr.stats.sampling_rate)
        p_is_defined = True
        

    data = tr.data

    size = 60
    input_dataset = np.zeros(shape=(0, size))
    current_index = 0

    while current_index < len(data):
        if current_index + size > len(data):
            difference = len(data) - current_index
            x = data[current_index:current_index+difference]
            difference2 = size - difference
            x = np.append(x, np.zeros(difference2))
        else:
            x = data[current_index:current_index+size]

        input_dataset = np.vstack((input_dataset, x))

        current_index = current_index + 1

    input_dataset = np.reshape(input_dataset, (len(input_dataset), size, 1))
    output = [x[1] for x in cnn.predict(input_dataset)]
    output_running_mean = running_mean(output, size)
    guess = np.argmax(output_running_mean)


#     plt.figure(figsize=(50,10))
    plt.plot(data)
#     plt.plot(tr.times(), data)

    if p_is_defined:
        plt.axvline(x=picker_p , color='red')
        
    plt.axvline(x=guess , color='green')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')

    plt.show()
#     plt.savefig('/home/irene/Documents/Resultados/'+tr.stats.station+'.png')

In [36]:
import os 

base_path = '/home/irene/Documents/Datos-Tesis/Estaciones/'

file = os.path.join(base_path, 'PEZE.HHZ.2018.337.16.57.18.sac')

print(file)
# picker_p_predict()

/home/irene/Documents/Datos-Tesis/Estaciones/PEZE.HHZ.2018.337.16.57.18.sac


In [ ]:
picker_p_predict(file)